<a href="https://colab.research.google.com/github/Sunnnyyy16/EmoSupportAI/blob/sunny/mental_health_finetune_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT fine-tuning with mental health conversation dataset
### Reference

* [source code](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb)
* [documentation guide](https://platform.openai.com/docs/guides/fine-tuning)
* [API reference](https://platform.openai.com/docs/api-reference/fine-tuning).


## Setup


In [ ]:
# make sure to use the latest version of the openai python package
!pip install --upgrade --quiet openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

# Set your API key directly here
OPENAI_API_KEY = "COPY OPENAPI KEY HERE"

client = openai.OpenAI(
    api_key=OPENAI_API_KEY
)

In [ ]:
## mental health counseling dataset 불러오기
import pandas as pd

df = pd.read_csv("hf://datasets/nbertagnolli/counsel-chat/20220401_counsel_chat.csv")
print(df.shape)
df.head()

(2775, 10)


,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386
2,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Shakeeta TorresFaith Based Mental Health Couns...,https://counselchat.com/therapists/shakeeta-to...,Absolutely not. I strongly recommending worki...,2,3071
3,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Noorayne ChevalierMA, RP, CCC, CCAC, LLP (Mich...",https://counselchat.com/therapists/noorayne-ch...,Let me start by saying there are never too man...,2,2643
4,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Toni Teixeira, LCSWYour road to healing begins...",https://counselchat.com/therapists/toni-teixei...,I just want to acknowledge you for the courage...,1,256


In [ ]:
# null값 확인
df.isnull().sum()

,0
questionID,0
questionTitle,0
questionText,139
questionLink,0
topic,0
therapistInfo,0
therapistURL,0
answerText,26
upvotes,0
views,0


In [ ]:
# null값 삭제한 최종 df 생성
final_df = df.dropna(axis=0)
final_df.isnull().sum()

,0
questionID,0
questionTitle,0
questionText,0
questionLink,0
topic,0
therapistInfo,0
therapistURL,0
answerText,0
upvotes,0
views,0


In [ ]:
final_df.head()

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386
2,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Shakeeta TorresFaith Based Mental Health Couns...,https://counselchat.com/therapists/shakeeta-to...,Absolutely not. I strongly recommending worki...,2,3071
3,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Noorayne ChevalierMA, RP, CCC, CCAC, LLP (Mich...",https://counselchat.com/therapists/noorayne-ch...,Let me start by saying there are never too man...,2,2643
4,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Toni Teixeira, LCSWYour road to healing begins...",https://counselchat.com/therapists/toni-teixei...,I just want to acknowledge you for the courage...,1,256


In [ ]:
# df.shape[0]
len(final_df)

2612

## Data preparation


In [ ]:
system_message = "You are a supportive mental health therapists. You are to provide supportive responses to people who suffer from mental health issues."


def create_user_message(row):
    return f"Title: {row['questionTitle']}\n\nquestionTitle: {row['questionText']}\n\nquestionText: "


def prepare_example_conversation(row):
    return {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": create_user_message(row)},
            {"role": "assistant", "content": row["answerText"]},
        ]
    }


pprint(prepare_example_conversation(final_df.iloc[0]))

{'messages': [{'content': 'You are a supportive mental health therapists. You '
                          'are to provide supportive responses to people who '
                          'suffer from mental health issues.',
               'role': 'system'},
              {'content': 'Title: Do I have too many issues for counseling?\n'
                          '\n'
                          'questionTitle: I have so many issues to address. I '
                          'have a history of sexual abuse, I’m a breast cancer '
                          'survivor and I am a lifetime insomniac.    I have a '
                          'long history of depression and I’m beginning to '
                          'have anxiety. I have low self esteem but I’ve been '
                          'happily married for almost 35 years.\n'
                          '   I’ve never had counseling about any of this. Do '
                          'I have too many issues to address in counseling?\n'
         

In [ ]:
# use the first 100 rows of the dataset for training
training_df = final_df.loc[0:len(final_df)*0.8] # 학습데이터: 80% 사용

# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a supportive mental health therapists. You are to provide supportive responses to people who suffer from mental health issues.'}, {'role': 'user', 'content': 'Title: Do I have too many issues for counseling?\n\nquestionTitle: I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?\n\nquestionText: '}, {'role': 'assistant', 'content': 'It is very common for\xa0people to have multiple issues that they want to (and need to) address in counseling.\xa0 I have had clients ask that same question and through more exploration, there is often an underlying fear that they\xa0 "can\'t be helped" or that they will "be too much for 

In [ ]:
validation_df = final_df.loc[len(final_df)*0.8:]   # validation data: 20% 사용
validation_data = validation_df.apply(
    prepare_example_conversation, axis=1).tolist()

In [ ]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [ ]:
training_file_name = "tmp_mental_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_mental_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

This is what the first 5 lines of our training `.jsonl` file look like:


In [ ]:
# print the first 5 lines of the training file
!head -n 5 tmp_mental_finetune_training.jsonl

{"messages": [{"role": "system", "content": "You are a supportive mental health therapists. You are to provide supportive responses to people who suffer from mental health issues."}, {"role": "user", "content": "Title: Do I have too many issues for counseling?\n\nquestionTitle: I have so many issues to address. I have a history of sexual abuse, I\u2019m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I\u2019m beginning to have anxiety. I have low self esteem but I\u2019ve been happily married for almost 35 years.\n   I\u2019ve never had counseling about any of this. Do I have too many issues to address in counseling?\n\nquestionText: "}, {"role": "assistant", "content": "It is very common for\u00a0people to have multiple issues that they want to (and need to) address in counseling.\u00a0 I have had clients ask that same question and through more exploration, there is often an underlying fear that they\u00a0 \"can't be helped\" or that 

### Upload files


In [ ]:
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id


training_file_id = upload_file(training_file_name, "fine-tune")
validation_file_id = upload_file(validation_file_name, "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-AoLSNjDuaIvCqCyCQIuJin42
Validation file ID: file-LFOcppmBdHPuG3PJ5iGhPkhd


## Fine-tuning


In [ ]:
MODEL = "gpt-4o-2024-08-06"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=MODEL,
    suffix="mental-therapy",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-cHoNEQF4lUC4M3jXC6Me2cvd
Status: validating_files


#### Check job status


In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-cHoNEQF4lUC4M3jXC6Me2cvd
Status: validating_files
Trained Tokens: None


We can track the progress of the fine-tune with the events endpoint. You can rerun the cell below a few times until the fine-tune is ready.


In [ ]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-cHoNEQF4lUC4M3jXC6Me2cvd
Validating training file: file-AoLSNjDuaIvCqCyCQIuJin42 and validation file: file-LFOcppmBdHPuG3PJ5iGhPkhd


Now that it's done, we can get a fine-tuned model ID from the job:


In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError(
        "Fine-tuned model ID not found. Your job has likely not been completed yet."
    )

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-4o-2024-08-06:personal:mental-therapy:AJxRwGjC


## Inference


In [ ]:
# 데이터의 마지막 50개를 테스트로 사용
test_df = final_df.loc[-50:]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You are a supportive mental health therapists. You are to '
             'provide supportive responses to people who suffer from mental '
             'health issues.',
  'role': 'system'},
 {'content': 'Title: Do I have too many issues for counseling?\n'
             '\n'
             'questionTitle: I have so many issues to address. I have a '
             'history of sexual abuse, I’m a breast cancer survivor and I am a '
             'lifetime insomniac.    I have a long history of depression and '
             'I’m beginning to have anxiety. I have low self esteem but I’ve '
             'been happily married for almost 35 years.\n'
             '   I’ve never had counseling about any of this. Do I have too '
             'many issues to address in counseling?\n'
             '\n'
             'questionText: ',
  'role': 'user'}]


In [ ]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

No one has too many issues for counseling.  In fact, the more issues you have, the more you can benefit from counseling.  A good therapist will help you prioritize your issues and work on them one at a time.  You will be amazed at how much better you will feel after you start working on your issues.  You will also be amazed at how much your issues are connected to each other.  For example, your history of sexual abuse is probably connected to your low self-esteem and your depression.  Your breast cancer is probably connected to your anxiety and your insomnia.  By working on one issue, you will be working on all of them.  You will be healing your whole self.  You deserve to heal and be happy.  You have already survived so much.  You are a strong and resilient person.  You can do this.  You are not alone.  There is help and hope for you.
